# What would the leaderboard look like if the private test labels were inadvertently shuffled?

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

np.random.seed(0)

leaderboard = pd.read_csv('../input/final-lb-rsnamiccai-brain-tumor-classification/rsna-private.csv')
team_n = leaderboard.shape[0]
entries = np.array(leaderboard['Entries'], dtype=np.int32)

# Size of the public test set
public_n = 87
# Size of the complete test set
total_n = 400

In [ ]:
labels = np.random.random(total_n).round()
best_scores = np.empty(team_n, dtype=np.float32)

for team in range(team_n):
    public_scores = np.empty(entries[team], dtype=np.float32)
    private_scores = np.empty_like(public_scores)
    for entry in range(entries[team]):
        preds = np.random.random(total_n)
        public_auc = roc_auc_score(labels[:public_n], preds[:public_n])
        private_auc = roc_auc_score(labels[public_n:], preds[public_n:])
        public_scores[entry] = public_auc
        private_scores[entry] = private_auc

    # Get the private score for the entry that scored the best on public LB
    idx = np.argmax(public_scores)
    best_scores[team] = round(private_scores[idx], 5)

In [ ]:
# Get the top scores in the private LB
sorted_scores = -np.sort(-best_scores)
simulated_top_ten = sorted_scores[:10]
actual_top_ten = leaderboard['Score'].values[:10]
print(f'Simulated top 10: {simulated_top_ten}')
print(f'Actual top 10   : {actual_top_ten}')

![](https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/120/apple/285/thinking-face_1f914.png)

In [ ]:
import seaborn as sns
data = pd.DataFrame({'actual':leaderboard['Score'].values, 'simulated':best_scores})
sns.histplot(data, kde=True)

![](https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/120/apple/285/relieved-face_1f60c.png)

## While the top scores on the actual leaderboard seem to line up with that of the simulated leaderboard, the score distributions are different.